#Create a Question Component - creates JAVA problems for practicing and evaluates and provides feedback for the same.

In [11]:
import os

from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.llms.cohere import Cohere

# Initialize Cohere LLM
cohere_llm = Cohere(cohere_api_key="9rZaWYoS4JsQMcOMZjozd66X0DqbStcRVe8LimVQ", model="command-r-plus")

# List of 10 topics for problem generation
topics = [
    "Inheritance and Polymorphism",
    "Encapsulation and Abstraction",
    "Interfaces and Abstract Classes",
    "Collections Framework",
    "Generics in Java",
    "Exception Handling",
    "Lambda Expressions",
    "Threads and Concurrency",
    "Design Patterns (e.g., Singleton, Factory)",
    "Java Stream API"
]

# Set up the retrieval function (simply returning the topic for simplicity)
def retrieve_relevant_info(query):
    """
    Simulate retrieval of related information (for now, just returning the topic).
    """
    return f"Relevant information for the topic '{query}'"

# Problem Generator Module (RAG-Enhanced)
def generate_problem(topic):
    """
    Generates a detailed Java programming problem in the specified OOP topic, enriched with relevant context.
    """
    # Retrieve related information
    relevant_info = retrieve_relevant_info(topic)

    # Create the problem prompt enriched with retrieved info
    problem_prompt = PromptTemplate(
        input_variables=["topic", "relevant_info"],
        template=(
            "You are a Java programming instructor. Create a detailed programming problem in the area of {topic}. "
            "Use the following information to help design the problem:\n{relevant_info}\n"
            "Include a clear rubric, deliverables, and requirements. Make the problem challenging yet solvable for students."
        )
    )

    # Set up the problem generation chain
    problem_chain = LLMChain(llm=cohere_llm, prompt=problem_prompt)
    return problem_chain.run(topic=topic, relevant_info=relevant_info)

# Code Evaluator Module (RAG-Enhanced)
def evaluate_code(problem_statement, rubric, student_code):
    """
    Evaluates the student's Java code against the problem statement and rubric, enriched with additional context.
    """
    # Retrieve common mistakes related to the topic for context
    common_mistakes = retrieve_relevant_info("common mistakes")

    # Create the evaluation prompt enriched with retrieved info
    evaluation_prompt = PromptTemplate(
        input_variables=["problem_statement", "rubric", "student_code", "common_mistakes"],
        template=(
            "You are a Java programming evaluator. The problem statement is as follows:\n{problem_statement}\n\n"
            "Rubric for evaluation:\n{rubric}\n\n"
            "The student's code submission is:\n{student_code}\n\n"
            "Common mistakes for this topic include:\n{common_mistakes}\n\n"
            "Provide a detailed evaluation based on the rubric, analyzing the correctness, design, and any improvements needed."
        )
    )

    # Set up the evaluation chain
    evaluation_chain = LLMChain(llm=cohere_llm, prompt=evaluation_prompt)
    return evaluation_chain.run(problem_statement=problem_statement, rubric=rubric, student_code=student_code, common_mistakes=common_mistakes)

# Feedback Generator Module (RAG-Enhanced)
def generate_feedback(evaluation):
    """
    Generates constructive feedback for the student based on the evaluation, incorporating relevant context.
    """
    # Retrieve relevant feedback tips (using common mistakes as a placeholder)
    relevant_feedback_tips = retrieve_relevant_info("common mistakes")

    # Create the feedback prompt enriched with retrieved tips
    feedback_prompt = PromptTemplate(
        input_variables=["evaluation", "relevant_feedback_tips"],
        template=(
            "You are a Java programming instructor. Based on the following evaluation, generate constructive feedback "
            "for the student. Focus on strengths, areas of improvement, and next steps for learning:\n{evaluation}\n"
            "Use the following tips to help guide the student's learning:\n{relevant_feedback_tips}"
        )
    )

    # Set up the feedback generation chain
    feedback_chain = LLMChain(llm=cohere_llm, prompt=feedback_prompt)
    return feedback_chain.run(evaluation=evaluation, relevant_feedback_tips=relevant_feedback_tips)

# Function to run the entire workflow and write results to a file
def gather_data():
    # Output file to store results
    output_file = "programming_problems_and_feedbacks.txt"

    # Create a rubric for evaluation
    rubric = "1. Correct use of inheritance (30 points).\n2. Implementation of polymorphic behavior (30 points).\n3. Code readability and comments (20 points).\n4. Test cases and correctness (20 points)."

    # Loop over each topic to generate problem, evaluate code, and provide feedback
    with open(output_file, "w") as file:
        for topic in topics:
            # Generate a programming problem
            problem = generate_problem(topic)
            file.write(f"Generated Problem for {topic}:\n{problem}\n\n")

            # Placeholder student code (to be evaluated)
            student_code = """
            class Animal {
                void sound() {
                    System.out.println("Animal makes a sound");
                }
            }

            class Dog extends Animal {
                @Override
                void sound() {
                    System.out.println("Dog barks");
                }
            }

            public class Main {
                public static void main(String[] args) {
                    Animal obj = new Dog();
                    obj.sound();
                }
            }
            """

            # Evaluate the student's code
            evaluation = evaluate_code(problem, rubric, student_code)
            file.write(f"Evaluation for {topic}:\n{evaluation}\n\n")

            # Generate feedback for the student
            feedback = generate_feedback(evaluation)
            file.write(f"Feedback for {topic}:\n{feedback}\n\n")

    print(f"Results have been written to {output_file}")

# Run the data gathering process
if __name__ == "__main__":
    gather_data()


Results have been written to programming_problems_and_feedbacks.txt
